In [18]:
import pandas as pd
from pandas import option_context
import numpy as np
import requests
import seaborn as sns
import datetime
import cliffsDelta as cd
import os

from tqdm import tqdm
from statsmodels.stats.inter_rater import fleiss_kappa,aggregate_raters
from scipy.stats import mannwhitneyu,shapiro,ttest_ind

In [19]:
QUERY_ROOT = "https://api.github.com"
TOKEN = os.getenv("GH_TOKEN") # write your GitHub API key here
HEADERS={'Authorization':'token '+TOKEN}

## Section 3

### Question 1

#### Write an automated Python script to identify the GitHub Apps in the list of accounts, by calling the GitHub Users API and extracting the required information to make this decision. 

#### More information: You can search for the "type" key in the obtianed JSON result, If it is "Bot", then it is an app, if it is "User", then it is an account.

In [20]:
# 1. Read the given .csv file into a pandas DataFrame (df).
# 2. Make sure there are 2 columns, one column name is "contributor" that has the name of the contributor, 
#    and another column name is "bothunter" that you will use later on in this assignment.
# 3. Display the DataFrame.

# YOUR CODE BELOW
df = pd.read_csv("microsoft_winget-pkgs_2/microsoft_winget-pkgs_2.csv")
print(df)

                          contributor bothunter
0                            lszomoru     Human
1                             piraces     Human
2                         CoolPlayLin     Human
3                          lutzroeder     Human
4                           TejasMate     Human
5                     dmitriy-abramov     Human
6                             twpayne     Human
7                         stacklokbot       Bot
8                          CHIZI-0618     Human
9                              Nifury     Human
10                              jjw24     Human
11                             exoosh     Human
12                       SpecterShell     Human
13               youtube-music-winget     Human
14               azure-pipelines[bot]       Bot
15                       msftbot[bot]       Bot
16                        msftrubengu     Human
17                          kevinboss     Human
18                        nvim-winget       Bot
19                           rozmansi   

In [21]:
# 1. Store the names of the contributors in a list using the code snippet: 'df.contributor.to_list()'
# 2. print the list

# YOUR CODE BELOW
names = df.contributor.to_list()
print(names)

['lszomoru', 'piraces', 'CoolPlayLin', 'lutzroeder', 'TejasMate', 'dmitriy-abramov', 'twpayne', 'stacklokbot', 'CHIZI-0618', 'Nifury', 'jjw24', 'exoosh', 'SpecterShell', 'youtube-music-winget', 'azure-pipelines[bot]', 'msftbot[bot]', 'msftrubengu', 'kevinboss', 'nvim-winget', 'rozmansi', 'stephengillie', 'ItzLevvie', 'stenya', 'ateoi', 'xiaoyang-sde', 'ath0mas', 'auanasgheps', 'MaanuelMM', 'marcauberer', 'JakubMatejka', 'azclibot', 'cafkbot-winget', 'itsAdee', 'pulumi-bot', 'robot-github-winget-datadog-agent', 'FaserF', 'zkochan', 'aaronliu0130', 'coolplaylinbot', 'podman-desktop-bot']


In [22]:
# 1. Using a for loop, iterate over each contributor in the list and query the GitHub Users API.
#    You can use "query = f'{QUERY_ROOT}/users/{contributor}'", where QUERY_ROOT is defined at the beginning of this notebook 
#    above and 'contributor' is each individul contributor from the list
# 2. Get the response using 'response = requests.get(query, headers=HEADERS)'.
# 3. convert the response to JSON using 'json_response = response.json()'.
# 4. Iterate over this JSON response and get the value of the 'type' key. If it is "Bot" then the contributor is an App, 
#    if "User" then the contributor is an account. You should boolean values to indicate if the contributor is an App (True) or User/Organisation (False)
# 5. Save these results in list of dictionary of the form [{'contributor': <contributor name>, 'app': <boolean value>}, {...}, {...}, {...}].
#    Lets call this list as "app_list_dict"
# 6. Finally convert this list of dictionary to DataFrame by writing 'pd.DataFrame.from_dict(app_dict)'
# 7. Display the DataFrame. This should have two columns - contributor and app

# YOUR CODE BELOW
app_list_dict = []
for contributor in names:
    query = f'{QUERY_ROOT}/users/{contributor}'
    response = requests.get(query, headers=HEADERS)
    json_response = response.json()
    is_app = json_response['type'] == "Bot"
    app_list_dict.append({'contributor':contributor, 'app':is_app})
app_df = pd.DataFrame.from_dict(app_list_dict)
print(app_df)

                          contributor    app
0                            lszomoru  False
1                             piraces  False
2                         CoolPlayLin  False
3                          lutzroeder  False
4                           TejasMate  False
5                     dmitriy-abramov  False
6                             twpayne  False
7                         stacklokbot  False
8                          CHIZI-0618  False
9                              Nifury  False
10                              jjw24  False
11                             exoosh  False
12                       SpecterShell  False
13               youtube-music-winget  False
14               azure-pipelines[bot]   True
15                       msftbot[bot]   True
16                        msftrubengu  False
17                          kevinboss  False
18                        nvim-winget  False
19                           rozmansi  False
20                      stephengillie  False
21        

### Question 2

#### Add an app column in the CSV file to store this information. You can write True if it is an app and False if it is not.


In [115]:
# Merge the app DataFrame to df by writing 'pd.merge(df, <app df>, on='contributor'). This is similar to SQL join on primary key 'contributor'.
# The resultant df should have 3 columns - contributor, bothunter and app.

# YOUR CODE BELOW
#df = pd.merge(df, app_df, on='contributor')
print(df)


                          contributor bothunter
0                            lszomoru     Human
1                             piraces     Human
2                         CoolPlayLin     Human
3                          lutzroeder     Human
4                           TejasMate     Human
5                     dmitriy-abramov     Human
6                             twpayne     Human
7                         stacklokbot       Bot
8                          CHIZI-0618     Human
9                              Nifury     Human
10                              jjw24     Human
11                             exoosh     Human
12                       SpecterShell     Human
13               youtube-music-winget     Human
14               azure-pipelines[bot]       Bot
15                       msftbot[bot]       Bot
16                        msftrubengu     Human
17                          kevinboss     Human
18                        nvim-winget       Bot
19                           rozmansi   

### Question 3

#### Report on the total number of GitHub Apps, and User accounts present in the list of accounts of your dataset.

In [24]:
df.to_csv("microsoft_winget-pkgs_2/microsoft_winget-pkgs_2_app.csv",index=False)

## Section 4

### Question 1

#### Write an automated script to report on the prediction of the GitHub accounts in your dataset, using BoDeGHa and RABBIT bot identification tools. For each tool, your script should add an extra column to the CSV ﬁle of your dataset, reporting the prediction (bot, human, unknown) of the considered tool on the considered account.

#### More information:
You can write an automated script to execute BoDeGHa and RABBIT, and save the results in a DataFrame. For BoDeGHa, you can ask the tool to give the output in CSV format, save it as a csv file and read it in a DataFrame. For RABBIT, you can use its internal argument to store the results in a .csv file and read it in a DataFrame. Remember to read their Documentation (Readme.md) for their input arguments and example outputs before using these tools.  
Similar to what you did in Section 2 - Question 1 and 2, add an additional column named 'bodegha' to store the predictions 
given by BoDeGHa and add another column called 'rabbit' to store the preditions given by RABBIT. 

In [25]:
# Read the .csv file that has the predicitons given by BoDeGHa.

# YOUR CODE BELOW
bdf = pd.read_csv("microsoft_winget-pkgs_2/bodegha_predictions.csv")
bdf = bdf.rename(columns={"account":"contributor","prediction":"bodegha"})

print(bdf)


                          contributor bodegha
0                          CHIZI-0618     Bot
1                         CoolPlayLin     Bot
2                           MaanuelMM     Bot
3                              Nifury     Bot
4                        SpecterShell     Bot
5                           TejasMate     Bot
6                               ateoi     Bot
7                         auanasgheps     Bot
8                azure-pipelines[bot]     Bot
9                      cafkbot-winget     Bot
10                     coolplaylinbot     Bot
11                            itsAdee     Bot
12                          kevinboss     Bot
13                           lszomoru     Bot
14                         lutzroeder     Bot
15                        marcauberer     Bot
16                        nvim-winget     Bot
17                         pulumi-bot     Bot
18  robot-github-winget-datadog-agent     Bot
19                        stacklokbot     Bot
20                            twpa

In [26]:
# merge it to df - Now your df should have the following four columns - contributor, bothunter, app, bodegha

# YOUR CODE BELOW
bdf_m = pd.merge(df,bdf, on="contributor", how="left")
bdf_m["bodegha"] = bdf_m.bodegha.fillna("Unknown")


print(bdf_m)

                          contributor bothunter  bodegha
0                            lszomoru     Human      Bot
1                             piraces     Human  Unknown
2                         CoolPlayLin     Human      Bot
3                          lutzroeder     Human      Bot
4                           TejasMate     Human      Bot
5                     dmitriy-abramov     Human  Unknown
6                             twpayne     Human      Bot
7                         stacklokbot       Bot      Bot
8                          CHIZI-0618     Human      Bot
9                              Nifury     Human      Bot
10                              jjw24     Human  Unknown
11                             exoosh     Human    Human
12                       SpecterShell     Human      Bot
13               youtube-music-winget     Human      Bot
14               azure-pipelines[bot]       Bot      Bot
15                       msftbot[bot]       Bot  Unknown
16                        msftr

In [27]:
# Read the predictions given by RABBIT from the corresponding .csv file 

# YOUR CODE BELOW
rdf = pd.read_csv("microsoft_winget-pkgs_2/rabbit.csv")
rdf = rdf.rename(columns={"account":"contributor"})
rdf

,Unnamed: 0,contributor,prediction,confidence
0,0,lszomoru,bot,0.930
1,1,piraces,human,1.000
2,2,CoolPlayLin,human,0.909
3,3,lutzroeder,human,0.948
4,4,TejasMate,bot,0.763
5,5,dmitriy-abramov,human,0.991
6,6,twpayne,human,0.999
7,7,stacklokbot,bot,0.965
8,8,CHIZI-0618,human,0.985
9,9,Nifury,human,0.999


In [28]:
# merge it to df - Now your df should have the following five columns - contributor, bothunter, app, bodegha, rabbit

# YOUR CODE BELOW
rdf_m = pd.merge(bdf_m,rdf, on="contributor", how="left")
rdf_m = rdf_m.drop('Unnamed: 0', axis=1)
rdf_m = rdf_m.drop('confidence', axis=1)
rdf_m = rdf_m.rename(columns={"prediction":"rabbit"})
print(rdf_m)

                          contributor bothunter  bodegha   rabbit
0                            lszomoru     Human      Bot      bot
1                             piraces     Human  Unknown    human
2                         CoolPlayLin     Human      Bot    human
3                          lutzroeder     Human      Bot    human
4                           TejasMate     Human      Bot      bot
5                     dmitriy-abramov     Human  Unknown    human
6                             twpayne     Human      Bot    human
7                         stacklokbot       Bot      Bot      bot
8                          CHIZI-0618     Human      Bot    human
9                              Nifury     Human      Bot    human
10                              jjw24     Human  Unknown    human
11                             exoosh     Human    Human    human
12                       SpecterShell     Human      Bot      bot
13               youtube-music-winget     Human      Bot      bot
14        

In [29]:
# Replace the prediction result given by rabbit from 'bot' to 'Bot', 'app' to 'Bot', 'human' to 'Human', 'unknown' to 'Unknown' to maintain consistency

# YOUR CODE BELOW
rdf_m["rabbit"] = rdf_m.rabbit.replace({'bot' : 'Bot', 'app' : 'Bot', 'human' : 'Human', 'unknown' : 'Unknown' })
print(rdf_m)


                          contributor bothunter  bodegha   rabbit
0                            lszomoru     Human      Bot      Bot
1                             piraces     Human  Unknown    Human
2                         CoolPlayLin     Human      Bot    Human
3                          lutzroeder     Human      Bot    Human
4                           TejasMate     Human      Bot      Bot
5                     dmitriy-abramov     Human  Unknown    Human
6                             twpayne     Human      Bot    Human
7                         stacklokbot       Bot      Bot      Bot
8                          CHIZI-0618     Human      Bot    Human
9                              Nifury     Human      Bot    Human
10                              jjw24     Human  Unknown    Human
11                             exoosh     Human    Human    Human
12                       SpecterShell     Human      Bot      Bot
13               youtube-music-winget     Human      Bot      Bot
14        

In [30]:
rdf_m.to_csv("microsoft_winget-pkgs_2/prediction_tools.csv")

### Question 2

#### Add an extra column to the CSV ﬁle called 'manual' that has the label determined by you. For each account, of your own opinion on its nature (bot, human or unknown only in case of doubt) on the basis of a manual veriﬁcation of the contributor’s activities in the repository through the GitHub UI and through the GitHub Events API https://api.github.com/users/username/events, and taking into account the prediction made by the bot identification tools.

#### More information:
For each contributor, look at their activities in GitHub UI, look at their latest events using GitHub Events API - https://api.github.com/users/<contributor\>/events, to make a decision on their type. 

In addition, you should also write a very small reason why do you think they are bot or human (e.g., same activity at regular interval so Bot, files committed and code modified looks like Human, comments look like Human, comment look like Bot, test report looks automated so Bot, and so on....). This might be useful to answer the next question as well.

In [31]:
# Write your manual decision here
# example : 
# manual_list_dict = [{'contributor': 'tommens', 'manual': 'human'}, {....}, {....}, {....}]
# pd.DataFrame.from_dict(manual_list_dict)

# YOUR CODE BELOW


In [32]:
# merge it with df of predictions. Now your df should have the following six columns
# contributor, bothunter, app, bodegha, rabbit, manual

# YOUR CODE BELOW


### Question 3

#### Study and report the purpose of these identified bots and humans in the repository assigned to you. 

For example, the purpose can be based on but not limited to - 

the type of activities that they are performing (releasing a version on every Sunday, updating the documentation), type of comments they are posting (reviewing code, test summary report), when are they triggered (when a new PR is created, when someone in the project ask the bot to merge the code) and so on. \textbf{Note:} The purpose can be other than what is provided in examples above. 


### Question 4

#### Create a column named 'type' to the CSV ﬁle and write your ﬁnal decision on the type of contributor. The ﬁnal decision on their type is the prediciton that majority of the raters predicted (you can ignore the 'Unknown'), in the case of contradiction, you can finalise it as Human.
#### Case 1: If two raters has mentioned a contributor as Bot and others mentioned it as 'Unknown', then the type of the account is Bot.  
#### Case 2: If 3 raters give 'Unknown' to a contributor then the prediction given by the fourth rater is final.
#### Case 3: If 2 raters give the prediciton as 'Bot' and 2 raters give it as 'Human', then the final decision is 'Human'
#### In this step, GitHub Apps should be marked as App and not as a Bot.

In [33]:
# Make a final decision on the type based on the predictions given by bothunter, bodegha, bothunter and your manual labelling. 
# Now your df should contain seven columns - contributor, bothunter, app, bodegha, rabbit, manual and type

# YOUR CODE BELOW
for ind in rdf_m.index:
    print(rdf_m['bothunter'][ind], rdf_m['bodegha'][ind], rdf_m['rabbit'][ind])

Human Bot Bot
Human Unknown Human
Human Bot Human
Human Bot Human
Human Bot Bot
Human Unknown Human
Human Bot Human
Bot Bot Bot
Human Bot Human
Human Bot Human
Human Unknown Human
Human Human Human
Human Bot Bot
Human Bot Bot
Bot Bot Bot
Bot Unknown Bot
Human Unknown Human
Human Bot Human
Bot Bot Unknown
Human Unknown Human
Bot Human Bot
Human Human Human
Human Unknown Human
Human Bot Bot
Human Unknown Human
Human Unknown Human
Human Bot Human
Human Bot Human
Human Bot Human
Human Unknown Human
Bot Unknown Bot
Human Bot Bot
Human Bot Human
Bot Bot Bot
Human Bot Bot
Human Human Human
Human Bot Human
Human Human Human
Bot Bot Bot
Bot Unknown Human


### Question 5 (a)

#### Compute and report the Fleiss Kappa interrater agreement score between the labels computed by all bot identiﬁcation tools.

##### For example:

converted_df = aggregate_raters(np.array(df[['rater1','rater2']]))  
kappa = fleiss_kappa(converted_df[0])  
print('kappa')  

#### a. between bothunter and rabbit

In [34]:
converted_df = aggregate_raters(np.array(rdf_m[['bothunter','rabbit']]))
kappa = fleiss_kappa(converted_df[0])
print(kappa) 

0.47329919531821535


#### b. between bothunter and bodegha

In [35]:
converted_df = aggregate_raters(np.array(rdf_m[['bothunter','bodegha']]))
kappa = fleiss_kappa(converted_df[0])
print(kappa) 

-0.26016260162601634


#### c. between bodegha and rabbit

In [36]:
converted_df = aggregate_raters(np.array(rdf_m[['bodegha','rabbit']]))
kappa = fleiss_kappa(converted_df[0])
print(kappa) 

-0.009591115598182801


#### d. between bothunter, bodegha and rabbit

In [37]:
converted_df = aggregate_raters(np.array(rdf_m[['bodegha','rabbit','bothunter']]))
kappa = fleiss_kappa(converted_df[0])
print(kappa) 

0.07076483202287348


### Question 5 (b)
#### Report on
#### a. What is you interpretation from the kappa values (use the table provided in the description document)? what do you understand?

Bodegha lower de agreement in all case.
The best agreement is between bothunter and rabbit

#### b. Based on your previous analysis, which bot identification tool would you use, if you want to predict if a contributor is a human or bot? Motivate your answer

Bothunter:
It has biggest rate with rabbit but Bothunter detect obvious bot that have bot in their name



## Section 5

#### Query the GitHub Events API to get maximum possible events for each contributor (maximum is 300 events).  
#### You can use the following query: '{QUERY_ROOT}/users/{contributor}/events?per_page=100&page={page}' where page = 1 or 2 or 3  
#### As you did in earlier task, make a request to GitHub Events API with the required headers, convert the response to JSON, extract the following fields and store in a DataFrame:
Event id - json_response['event_id']  
Name of the contributor - json_response['actor']['login']  
Name of the repository - json_response['repository']['name']  
Type of event - json_response['type']  
Time of event - json_response['created_at']

#### Hint: Query the GitHubEvents API for all contributors. Append the obtained JSON result to a list of JSONs. Process one dictionary after other to get your required fields and store them in another list of dictionaries. Finally convert this dictionaty to DataFrame. OR You can directly convert the list of JSONs to DataFrame using pd.json_normalize() and select the required columns alone.

#### The resultant dataframe should have all the recent events performed by all the contributors and the following columns should be present in them - event_id, contributor, repo_name, event_type, created_at.


In [39]:
# YOUR CODE BELOW

data = {"Event_id":[],"Ccontributor":[],"Repository":[],"Event_type":[],"Event_time":[]}

for contributor in names:
    for page in range(1,4):
        query = f"{QUERY_ROOT}/users/{contributor}/events?per_page=100&page={page}"
        response = requests.get(query, headers=HEADERS)
        json_responses = response.json()
        for json_response in json_responses:
            data["Event_id"].append(json_response['id'])
            data["Ccontributor"].append(json_response['actor']['login'])
            data["Repository"].append(json_response['repo']['name'])
            data["Event_type"].append(json_response['type'])
            data["Event_time"].append(json_response['created_at'])



In [43]:
edf = pd.DataFrame(data)
edf = edf.rename(columns={"Event_id":"event_id","Ccontributor":"contributor","Repository":"repository","Event_type":"event_type","Event_time":"event_time"})
edf
edf.to_csv("./microsoft_winget-pkgs_2/event.csv")

### Question 1
#### Group each event into the following four categories

Issues: IssueCommentEvent, IssuesEvent  
Pull Requests: PullRequestEvent, PullRequestReviewCommentEvent  
Commits: CommitCommentEvent, PushEvent  
Repository: CreateEvent, DeleteEvent, ForkEvent, GollumEvent, MemberEvent, PublicEvent, ReleaseEvent, SponsorshipEvent, WatchEvent  

#### Hint:
1. Add a column called event_group that mentions which event group does that event belong to. Each event (row) should correspond to an event group.  
2. Then perform groupby on ['contributor','event_group'],  
3. use .agg (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html) to count the number of events performed by each contributor in each group,  
4. use pivot with the required arguments (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html). An example is given [here](#pivot_example) in question 2(b), and  
5. finally merge it with your 'type' field from the DataFrame you got in Section4

The final DataFrame should be looking like the following  
![](event_group.png)

In [44]:
# In the resultant DataFrame - each row should corespond to a contributor, and the columns should have all the event groups  
# and the type of the contributor (that you decided in the previous DataFrame) and the values should be the number 
# of events of that event event group the contriubutor has performed.

issue_group = ['IssueCommentEvent', 'IssuesEvent']
pr_group = ['PullRequestEvent', 'PullRequestReviewCommentEvent']
commit_group = ['CommitCommentEvent', 'PushEvent']
repo_group = ['CreateEvent', 'DeleteEvent', 'ForkEvent', 'GollumEvent', 'MemberEvent', 'PublicEvent', 'ReleaseEvent', 'SponsorshipEvent', 'WatchEvent']

In [119]:
# YOUR CODE BELOW
total = {}
for ind in edf.index:
    cont = total.setdefault(edf['contributor'][ind],{"contributor":edf['contributor'][ind]})
    typ = edf['event_type'][ind]
    if typ in issue_group:
        cont.setdefault("issue",0)
        cont["issue"] +=1
    elif typ in pr_group:
        cont.setdefault("PR",0)
        cont["PR"] +=1
    elif typ in commit_group:
        cont.setdefault("commit",0)
        cont["commit"] +=1
    elif typ in repo_group:
        cont.setdefault("repo",0)
        cont["repo"] +=1

tdf = pd.DataFrame()
for v in total.values():
    tdf = pd.concat([tdf, pd.DataFrame(v,index=[0])],ignore_index=True)
tdf = pd.merge(df, tdf, on='contributor')
tdf = tdf.rename(columns={"bothunter":"type"})
tdf.to_csv('./microsoft_winget-pkgs_2/event_count.csv')
tdf


,contributor,type,commit,repo,PR,issue
0,lszomoru,Human,114.0,11.0,30,73.0
1,piraces,Human,31.0,17.0,27,5.0
2,CoolPlayLin,Human,117.0,79.0,75,24.0
3,lutzroeder,Human,139.0,62.0,32,52.0
4,TejasMate,Human,138.0,84.0,71,7.0
5,dmitriy-abramov,Human,5.0,6.0,6,NaN
6,twpayne,Human,100.0,75.0,53,52.0
7,stacklokbot,Bot,187.0,16.0,23,NaN
8,CHIZI-0618,Human,36.0,134.0,5,21.0
9,Nifury,Human,46.0,33.0,22,14.0


In [58]:
# YOUR CODE BELOW


,lszomoru,piraces,CoolPlayLin,lutzroeder,TejasMate,dmitriy-abramov,twpayne,stacklokbot,CHIZI-0618,Nifury,...,azclibot,cafkbot-winget,itsAdee,pulumi-bot,robot-github-winget-datadog-agent,FaserF,zkochan,aaronliu0130,coolplaylinbot,podman-desktop-bot
commit,114,31,117,139,138,5.0,100,187.0,36,46,...,110,12.0,60,25,27.0,63,168,37,233,13.0
repo,11,17,79,62,84,6.0,75,16.0,134,33,...,60,22.0,104,35,18.0,31,51,47,33,14.0
PR,30,27,75,32,71,6.0,53,23.0,5,22,...,53,12.0,37,37,16.0,23,39,28,29,21.0
issue,73,5,24,52,7,NaN,52,NaN,21,14,...,76,NaN,3,48,NaN,45,31,144,5,NaN


### Question 2 (a)

#### Compute the median number of events per event group for Bot+Apps and Humans and write in DataFrame.

Row should correspond to type (Bot_App and Human), Column should have Event group name and the values should be the median value of Bot_App or Human for that particular event group. An example is given below

In [120]:
# For example:
medians = [{'event_group': 'event_group1', 'median': 'val1', 'type': 'Bot_app'}, 
           {'event_group': 'event_group1', 'median': 'val2', 'type': 'Human'},
           {'event_group': 'event_group2', 'median': 'val3', 'type': 'Bot_app'},
           {'event_group': 'event_group2', 'median': 'val4', 'type': 'Human'},
           {'event_group': 'event_group3', 'median': 'val5', 'type': 'Bot_app'},
           {'event_group': 'event_group3', 'median': 'val6', 'type': 'Human'},
           {'event_group': 'event_group4', 'median': 'val7', 'type': 'Bot_app'},
           {'event_group': 'event_group5', 'median': 'val8', 'type': 'Human'}]
df_medians = pd.DataFrame.from_dict(medians)
df_medians

,event_group,median,type
0,event_group1,val1,Bot_app
1,event_group1,val2,Human
2,event_group2,val3,Bot_app
3,event_group2,val4,Human
4,event_group3,val5,Bot_app
5,event_group3,val6,Human
6,event_group4,val7,Bot_app
7,event_group5,val8,Human


In [ ]:
# YOUR CODE BELOW

### Question 2 (b)

Plot a heatmap of the DataFrame using seaborn - 
1. First convert the dataframe to the required format using pivot, example is given below
2. plot using seaborn - sns.heatmap(df_medians, annot=True, vmin=0, vmax=300, cmap="crest"). More details: https://seaborn.pydata.org/generated/seaborn.heatmap.html)

#### pd.pivot example:
<a id='pivot_example'></a>

In [ ]:
# Main DataFrame
medians = [{'event_group': 'event_group1', 'median': 'val1', 'type': 'Bot_app'}, 
           {'event_group': 'event_group1', 'median': 'val2', 'type': 'Human'},
           {'event_group': 'event_group2', 'median': 'val3', 'type': 'Bot_app'},
           {'event_group': 'event_group2', 'median': 'val4', 'type': 'Human'},
           {'event_group': 'event_group3', 'median': 'val5', 'type': 'Bot_app'},
           {'event_group': 'event_group3', 'median': 'val6', 'type': 'Human'},
           {'event_group': 'event_group4', 'median': 'val7', 'type': 'Bot_app'},
           {'event_group': 'event_group5', 'median': 'val8', 'type': 'Human'}]
df_medians = pd.DataFrame.from_dict(medians)
df_medians

In [ ]:
# pivot the main DataFrame
df_medians.pivot(index='type', columns='event_group', values='median')#.reset_index().rename_axis(None,axis=1)

In [ ]:
# YOUR CODE BELOW

In [ ]:
# YOUR CODE BELOW


#### What is the difference that you observe between Bots+Apps and Humans?

#### What is the differnce that you observe between Event groups?

#### What is the difference that you observe between Bots+Apps and Humans and Event groups all considering at the same time?

### Question 3

#### Create box plots or boxen plots or violin plots or a combination of these to visualise the distribution of number of events in each event group. 
#### For more information you can visit - https://seaborn.pydata.org/tutorial/categorical.html#categorical-tutorial  
#### For violin plots - https://seaborn.pydata.org/generated/seaborn.violinplot.html  
#### You should highlight the data points that correspond to bots using a stripplot in seaborn. https://seaborn.pydata.org/tutorial/categorical.html#categorical-tutorial  
#### Interpret the results of the visualisation.

In [ ]:
# YOUR CODE BELOW - Visualize number of events from pull request event group


In [ ]:
# YOUR CODE BELOW - Visualise number of events from issue event group


In [ ]:
# YOUR CODE BELOW - Visualise number of events from commit event group


In [ ]:
# YOUR CODE BELOW - Visualise number of events from repo event group


### Question 4.1

#### Statistical identify whether number of events in each event group is normally distributed or not.
#### Null hypothesis - $H_0$: Sample comes from the data that has normal distribution.
#### Use Shapiro-Wilk test for this purpose. Use the p-value with a threshold of 0.05 to determine whether $H_0$ can be rejected with statistical significance or not. 

#### Use shapiro from scipy.stats to perform this test (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html)

In [ ]:
# YOUR CODE BELOW

### Question 4.2
#### Determine if there is any statistical diﬀerence in number of events between the identified event groups. Perform this test on all pairs of event groups. 
#### Null hypothesis - $H_0$: Any two event group come from the same population.
#### If $H_0$ is rejected in Shapiro-Wilk test (at least for one of the two event groups considered for test), use the Mann-Whitney U statistical test for this purpose. If $H_0$ is not rejected in Shapiro-Wilk test, use the independent t-test for this purpose.  
#### In any case, use the p-value with a threshold of 0.01 to determine whether $H_0$ can be rejected with statistical significance. 

#### Use mannwhitneyu from scipy.stats to perform Mann-Whitney U test (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html) 
#### or 
#### ttest_ind from scipy.stats to perform independent t-test https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html.

#### More information:

You can pass the following arguments to mannwhitneyu from scipy.stats - (method='exact', nan_policy='omit'). For ttest_ind you can use - (nan_policy='omit')

In [ ]:
# YOUR CODE BELOW

### Question 5
#### Each time you reject the null hypothesis $H_0$, quantify the effect size of the diﬀerence between the groups using cliﬀ’s delta ($\delta$). 
#### To calculate cliﬀ’s delta, you can pass the list of values to cliﬀ delta.py file given in the repository. E.g., cliffsDelta.cliffsDelta(list of values). This will return the effect size and its interpretation (negligible, small, medium, large).

In [ ]:
# YOU CODE BELOW

### Qusetion 6

#### Create violinplots to visualise the distributions for the following metrics and answer the questions that follow it. You should highlight the data points corresponding to bots using a stripplot in seaborn.

#### Hint:

1. Merge you DataFrame of event with the type of contributor that you identified in section 3.
2. Write a function that you will use for all the seaborn plots. This way you can reduce duplication and update it easily.

In [ ]:
# YOUR CODE BELOW - function for seaborn plot


#### Total number of events

In [ ]:
# YOUR CODE BELOW

#### Number of event types

In [57]:
# YOUR CODE BELOW

#### Mean number of events per event type

In [ ]:
# YOUR CODE BELOW

#### Mean number of events per event group

In [ ]:
# YOUR CODE BELOW

#### Gini coeﬃcient of time between consecutive event

In [ ]:
def time_diffs(time_stamps):
    diff_time_stamps = np.array([])
    for i in range(len(time_stamps)-1):
        diff = (time_stamps.iloc[i+1] - time_stamps.iloc[i])/pd.to_timedelta('1 second')
        diff_time_stamps = np.append(diff_time_stamps, [diff])
    return(diff_time_stamps)

In [ ]:
def gini(time_stamps):
    '''
    args: time_stamps - Series of time stamps for a contributor
    
    return: g_coeff - float that gives gini coefficient
    
    method: calculate the Gini coefficient of a numpy array
    '''
    array = time_diffs(time_stamps)
    array = array[array!=0]
    if len(array) == 0:
        return np.float64(0.0)
    array = array.flatten()
    array = np.sort(array)
    index = np.arange(1, array.shape[0] + 1)
    n = array.shape[0]
    g_coeff = (np.sum((2 * index - n - 1) * array)) / (n * np.sum(array))
    
    return (g_coeff.round(4))

You can pass a Series of time-stamps to gini(time_stamps) function to get the Gini coeﬃcient
#### Hint:
1. Groupby the contributors on 'contributor' and use gini as .agg function on 'created_at'.
2. Merge the resultant DataFram with their type (the df that you got in section 4)
3. Make box/boxen/violin plots on those gini values

In [ ]:
# YOUR CODE BELOW


In [ ]:
# YOUR CODE BELOW


### Question 6.1

#### Do you ﬁnd any set of accounts having a diverging behaviour? (Hint: look for outliers.)

### Question 6.2

#### What is the type of the accounts that have diverging behaviour?

### Question 6.3

#### Do bots have different behaviour than apps? Motivate your answer

### Question 6.4

#### Do bots and apps have a diﬀerent behaviour from human accounts? (Hint: in the box plots ﬁnd where the bots and apps are located and find where the humans are located)

### Question 6.5

#### Do all bots (other than outliers) have the same behaviour? List the bots that have a diﬀerent behaviour. Mention your interpretation on the cause of this diverging behaviour compared to other bots?